# **IACEC Project: Random Forest on Small Imbalanced Datasets**

## **Group 07: G07**

### **Authors:**
- **Bruno Fernandes**: [up202108871](up202108871@up.pt)
- **Hugo Abelheira**: [up202409899](up202409899@up.pt)

## **Introduction**

### **Context**

This project was developed in for the course "Introduction to Algorithms and Data Structures" (IACEC) of the Master's in Data Science and Engineering (MDSE) at the Faculty of Engineering of the University of Porto (FEUP).

In this project, we aim to explore the Random Forest algorithm in the context of small imbalanced datasets. We will analyze the performance of the algorithm in this context and propose some strategies to improve it.

### **Issue At Hand**

We chose this topic because we believe that the Random Forest algorithm is a powerful tool for binary classification tasks, but it can be sensitive to imbalanced datasets. We want to understand how the algorithm behaves in this context and how we can improve its performance.

### **Dataset Characteristics**

The implementation is tested on datasets from the OpenML-CC18 collection with:
- Binary classification tasks
- Significant class imbalance (minority class < 20%)

### **Proposed Adjustments**

We propose, implement, and test SMOTE oversampling techniques instead of the bootstrap sampling used in the default Random Forest algorithm. This technique aims to balance the classes in the training set by generating synthetic samples of the minority class. By applying SMOTE during the training phase, we address the fundamental issue of class imbalance at the data level, allowing each tree in the Random Forest to learn from a more balanced dataset.

In our implementation, we focus on maximizing recall for the positive (minority) class, as in many real-world applications with imbalanced datasets, it is more important to identify positive cases correctly (minimize false negatives) even at the cost of increased false positives.



### **Imports**

In [ ]:
import time
import warnings
from collections import Counter

import matplotlib.pyplot as plt
import numpy as np
import openml
import pandas as pd
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.metrics import (ConfusionMatrixDisplay, auc,
                             classification_report, confusion_matrix,
                             roc_curve)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize

# ignore warnings
warnings.filterwarnings("ignore")

### **Getting the Imbalanced Datasets from OpenML-CC18**

The check_class_imbalance function is used to identify and list the most imbalanced datasets in the OpenML 99 collection, based on a specified threshold for class imbalance. This step allows us to select the datasets most suitable for the purpose of our problem, which involves addressing and correcting class imbalance.

In [ ]:
def check_class_imbalance(threshold=0.8):
    print("Loading OpenML Collection 99 datasets...")
    
    # Get datasets from OpenML Collection with ID 99
    datasets = openml.study.get_suite(99)  # Suite ID 99

    # List to store datasets with imbalance
    imbalanced_datasets = []

    print("Checking for class imbalance in OpenML Collection 99...\n")

    for dataset_id in datasets.data:
        try:
            # Download the dataset
            dataset = openml.datasets.get_dataset(dataset_id)
            X, y, _, attributes = dataset.get_data(
                target=dataset.default_target_attribute
            )

            # Ensure the target variable is categorical
            if not isinstance(y, pd.Categorical):
                y = pd.Categorical(y)

            # Calculate class distribution
            class_counts = y.value_counts()  # Absolute counts
            total = class_counts.sum()  # Total number of instances
            class_proportions = class_counts / total  # Normalized proportions
            max_class_proportion = class_proportions.max()

            # Check for imbalance
            if max_class_proportion >= threshold:
                imbalanced_datasets.append(
                    {
                        "Dataset ID": dataset_id,
                        "Dataset Name": dataset.name,
                        "Max Class Proportion": max_class_proportion,
                        "Class Distribution": class_proportions.to_dict(),
                    }
                )

        except Exception as e:
            print(f"Error processing dataset {dataset_id}: {e}")
            continue

    # Sort the imbalanced datasets by max class proportion in descending order
    imbalanced_datasets = sorted(
        imbalanced_datasets, key=lambda x: x["Max Class Proportion"], reverse=True
    )

    # Print results
    if imbalanced_datasets:
        print("Datasets with class imbalance greater than the threshold:\n")
        for dataset in imbalanced_datasets:
            print(f"Dataset ID: {dataset['Dataset ID']}")
            print(f"Dataset Name: {dataset['Dataset Name']}")
            print(f"Max Class Proportion: {dataset['Max Class Proportion']:.2%}")
            print("Class Distribution:")
            for cls, proportion in dataset["Class Distribution"].items():
                print(f"  {cls}: {proportion:.2%}")
            print()
    else:
        print("No datasets found with class imbalance greater than the threshold.")

    return imbalanced_datasets


# Run the script
imbalanced_datasets = check_class_imbalance(threshold=0.8)

#### **Choosing the Datasets for the Benchmark**

In [ ]:
imbalanced_datasets = [imbalanced_datasets[0], imbalanced_datasets[2], imbalanced_datasets[4], imbalanced_datasets[5]]

### **Random Forest Implementation**

Our implementation includes a parameter `smote` that allows choosing between the original bootstrap sampling and SMOTE oversampling.


In [ ]:
class TreeNode:
    def __init__(
        self, feature=None, threshold=None, left=None, right=None, *, value=None
    ):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value


class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    def gini(self, y):
        counts = np.bincount(y)
        probabilities = counts / len(y)
        return 1 - np.sum(probabilities**2)

    def best_split(self, X, y):
        best_gini = 1.0
        best_feature, best_threshold = None, None
        n_features = X.shape[1]

        for feature in range(n_features):
            X_feature = X[:, feature]
            sorted_indices = np.argsort(X_feature)
            X_sorted = X_feature[sorted_indices]
            y_sorted = y[sorted_indices]

            # Identify potential split points
            for i in range(1, len(y_sorted)):
                if y_sorted[i] != y_sorted[i - 1]:
                    threshold = (X_sorted[i] + X_sorted[i - 1]) / 2
                    left_mask = X_sorted < threshold
                    y_left = y_sorted[left_mask]
                    y_right = y_sorted[~left_mask]

                    if len(y_left) == 0 or len(y_right) == 0:
                        continue

                    gini_left = self.gini(y_left)
                    gini_right = self.gini(y_right)
                    gini = (len(y_left) * gini_left + len(y_right) * gini_right) / len(
                        y_sorted
                    )

                    if gini < best_gini:
                        best_gini = gini
                        best_feature = feature
                        best_threshold = threshold

        return best_feature, best_threshold

    def build_tree(self, X, y, depth=0):
        num_samples, _ = X.shape
        if (
            self.max_depth is not None and depth >= self.max_depth
        ) or num_samples < self.min_samples_split:
            leaf_value = Counter(y).most_common(1)[0][0]
            return TreeNode(value=leaf_value)

        feature, threshold = self.best_split(X, y)
        if feature is None:
            leaf_value = Counter(y).most_common(1)[0][0]
            return TreeNode(value=leaf_value)

        # Split the data
        left_indices = X[:, feature] < threshold
        X_left, y_left = X[left_indices], y[left_indices]
        X_right, y_right = X[~left_indices], y[~left_indices]

        if len(y_left) == 0 or len(y_right) == 0:
            leaf_value = Counter(y).most_common(1)[0][0]
            return TreeNode(value=leaf_value)

        # Recursively build the left and right subtrees
        left = self.build_tree(X_left, y_left, depth + 1)
        right = self.build_tree(X_right, y_right, depth + 1)
        return TreeNode(feature, threshold, left, right)

    def fit(self, X, y):
        self.root = self.build_tree(X, y)

    def predict_sample(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature] < node.threshold:
            return self.predict_sample(x, node.left)
        else:
            return self.predict_sample(x, node.right)

    def predict(self, X):
        return np.array([self.predict_sample(x, self.root) for x in X])


class RandomForest:
    def __init__(
        self,
        n_estimators=100,
        max_depth=None,
        min_samples_split=2,
        max_features="sqrt",
        smote=False,
        random_state=None,
    ):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.smote = smote
        self.random_state = random_state
        self.trees = []

    def bootstrap_sample(self, X, y):
        np.random.seed(self.random_state)
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, n_samples, replace=True)
        X_sample, y_sample = X[indices], y[indices]

        return X_sample, y_sample

    def fit(self, X, y):
        # Convert to NumPy arrays if they are pandas structures
        if isinstance(X, pd.DataFrame) or isinstance(X, pd.Series):
            X = X.values
        if isinstance(y, pd.DataFrame) or isinstance(y, pd.Series):
            y = y.values

        # Apply SMOTE if enabled
        if self.smote:
            sm = SMOTE(sampling_strategy=1, random_state=self.random_state)
            X, y = sm.fit_resample(X, y)

        self.trees = []
        n_features_total = X.shape[1]

        # Determine the number of features to consider at each split
        if self.max_features == "sqrt":
            max_features = int(np.sqrt(n_features_total))
        elif self.max_features == "log2":
            max_features = int(np.log2(n_features_total))
        elif isinstance(self.max_features, int):
            max_features = self.max_features
        elif self.max_features.lower() == "auto":
            max_features = n_features_total
        else:
            raise ValueError("RandomForest.fit: Invalid value for max_features")

        for i in range(self.n_estimators):
            tree = DecisionTree(
                max_depth=self.max_depth, min_samples_split=self.min_samples_split
            )
            if self.smote:
                X_sample, y_sample = X, y
            else:
                X_sample, y_sample = self.bootstrap_sample(X, y)

            # Select random subset of features
            features = np.random.choice(n_features_total, max_features, replace=False)
            
            # Fit the tree on the sampled data with selected features
            tree.fit(X_sample[:, features], y_sample)
            
            tree.features = features
            self.trees.append(tree)

    def predict(self, X):
        # Convert to NumPy array if it's a pandas structure
        if isinstance(X, pd.DataFrame) or isinstance(X, pd.Series):
            X = X.values

        tree_preds = []
        for i, tree in enumerate(self.trees):
            try:
                preds = tree.predict(X[:, tree.features])
                tree_preds.append(preds)
            except Exception as e:
                print(f"RandomForest.predict: Error predicting with tree {i+1}: {e}")
                raise e

        tree_preds = np.array(tree_preds).T  # Shape: (n_samples, n_estimators)
        y_pred = [Counter(row).most_common(1)[0][0] for row in tree_preds]
        return np.array(y_pred)


# Helper function
def convert_to_numeric(X, y):
    # Convert X to float
    if X.dtype == "O":
        X = X.astype(float)

    # Convert y to integer
    if y.dtype == "O":
        y = y.astype(int)

    return X, y

In [ ]:
def display_reports_side_by_side(
    y_true,
    y_pred_1,
    y_pred_2,
    titles=["Classification Report (Bootstrap)", "Classification Report (SMOTE)"],
    cmap="coolwarm",
):
    """
    Display two classification reports side by side with a shared colorbar.

    Parameters:
        y_true (array-like): True labels for both datasets.
        y_pred_1 (array-like): Predicted labels for the first dataset.
        y_pred_2 (array-like): Predicted labels for the second dataset.
        titles (list): Titles for each subplot. Default is ["Classification Report (Bootstrap)", "Classification Report (SMOTE)"].
        cmap (str): Colormap for the heatmaps. Default is "coolwarm".

    Returns:
        None: Displays the classification reports side by side.
    """
    # Set up the figure with two subplots side by side
    fig, axes = plt.subplots(1, 2, figsize=(16, 6), gridspec_kw={"wspace": 0.4})

    # Generate the first classification report and plot it
    report_dict_1 = classification_report(y_true, y_pred_1, output_dict=True)
    report_df_1 = (
        pd.DataFrame(report_dict_1)
        .transpose()
        .drop(["macro avg"], errors="ignore")
        .round(2)
    )
    heatmap_1 = sns.heatmap(
        report_df_1.iloc[:-1, :-1],
        annot=True,
        fmt=".2f",
        cmap=cmap,
        ax=axes[0],
        cbar=False,  # Disable individual colorbars
        linewidths=0.5,
        linecolor="gray",
    )
    axes[0].set_title(titles[0], fontsize=14, pad=10)
    axes[0].set_xlabel("Metrics", fontsize=12)
    axes[0].set_ylabel("Classes", fontsize=12)
    axes[0].tick_params(axis="x", labelrotation=45)

    # Generate the second classification report and plot it
    report_dict_2 = classification_report(y_true, y_pred_2, output_dict=True)
    report_df_2 = (
        pd.DataFrame(report_dict_2)
        .transpose()
        .drop(["macro avg"], errors="ignore")
        .round(2)
    )
    heatmap_2 = sns.heatmap(
        report_df_2.iloc[:-1, :-1],
        annot=True,
        fmt=".2f",
        cmap=cmap,
        ax=axes[1],
        cbar=False,  # Disable individual colorbars
        linewidths=0.5,
        linecolor="gray",
    )
    axes[1].set_title(titles[1], fontsize=14, pad=10)
    axes[1].set_xlabel("Metrics", fontsize=12)
    axes[1].tick_params(axis="x", labelrotation=45)

    # Add a shared colorbar
    # Create a new axis for the colorbar (on the right)
    cbar_ax = fig.add_axes([0.92, 0.3, 0.02, 0.4])  # [left, bottom, width, height]
    norm = plt.Normalize(
        vmin=min(
            report_df_1.iloc[:-1, :-1].min().min(),
            report_df_2.iloc[:-1, :-1].min().min(),
        ),
        vmax=max(
            report_df_1.iloc[:-1, :-1].max().max(),
            report_df_2.iloc[:-1, :-1].max().max(),
        ),
    )
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])  # Required for ScalarMappable
    fig.colorbar(sm, cax=cbar_ax)

    # Tight layout and display
    plt.tight_layout(rect=[0, 0, 0.9, 1])  # Adjust layout to fit colorbar
    plt.show()

In [ ]:
def evaluate_model(dataset, random_state=99):
    """
    Function to evaluate Random Forest model using Bootstrap and SMOTE
    with comparison plots for Confusion Matrix and ROC-AUC.

    Parameters:
        dataset (pd.DataFrame): Input dataset with features and target as the last column.
        random_state (int): Random seed for reproducibility.

    Returns:
        None: Displays the results in plots and prints execution times.
    """
    # Separate features and target variable into X and y, respectively
    X = dataset.drop(columns=[dataset.columns.values[-1]])
    y = dataset[dataset.columns.values[-1]]

    # Split data into train and test sets (stratified)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=random_state, stratify=y
    )

    #####################
    # --- Bootstrap Approach ---
    #####################
    start_time_bootstrap = time.time()

    rf_bootstrap = RandomForest(n_estimators=15, max_depth=5, smote=False, random_state=random_state)
    rf_bootstrap.fit(X_train, y_train)
    predictions_bootstrap = rf_bootstrap.predict(X_test)

    elapsed_bootstrap = time.time() - start_time_bootstrap

    # Metrics for Bootstrap
    cm_bootstrap = confusion_matrix(y_test, predictions_bootstrap)
    y_test_bin_bootstrap = label_binarize(y_test, classes=np.unique(y))
    predictions_bin_bootstrap = label_binarize(predictions_bootstrap, classes=np.unique(y))

    # Class distribution for Bootstrap
    y_train_counts = y_train.value_counts()

    #####################
    # --- SMOTE Approach ---
    #####################
    start_time_smote = time.time()

    sm = SMOTE(sampling_strategy=1, random_state=random_state)
    _, y_resampled = sm.fit_resample(X_train, y_train)

    rf_smote = RandomForest(n_estimators=15, max_depth=5, smote=True, random_state=random_state)
    rf_smote.fit(X_train, y_train)
    predictions_smote = rf_smote.predict(X_test)

    elapsed_smote = time.time() - start_time_smote

    # Metrics for SMOTE
    cm_smote = confusion_matrix(y_test, predictions_smote)
    y_test_bin_smote = label_binarize(y_test, classes=np.unique(y))
    predictions_bin_smote = label_binarize(predictions_smote, classes=np.unique(y))

    # --- Pie Charts for Class Distribution ---
    y_resampled_counts = pd.Series(y_resampled).value_counts()

    # Adjusting figure size for better visualization
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))  # Reduced figsize for smaller pie charts

    # Pie chart for class distribution before SMOTE
    axes[0].pie(
        y_train_counts,
        labels=y_train_counts.index,
        autopct='%1.1f%%',
        startangle=90,
        colors=sns.color_palette("muted"),
    )
    axes[0].set_title("Class Distribution before SMOTE", fontsize=12)

    # Pie chart for class distribution after SMOTE
    axes[1].pie(
        y_resampled_counts,
        labels=y_resampled_counts.index,
        autopct='%1.1f%%',
        startangle=90,
        colors=sns.color_palette("muted"),
    )
    axes[1].set_title("Class Distribution using SMOTE", fontsize=12)

    # Final layout adjustments
    plt.tight_layout()
    plt.show()

    # --- Confusion Matrices and ROC Curves ---
    _, axes = plt.subplots(2, 2, figsize=(18, 12))  # Increased figsize for larger matrices and curves

    # Plot Confusion Matrices
    disp_bootstrap = ConfusionMatrixDisplay(confusion_matrix=cm_bootstrap, display_labels=np.unique(y))
    disp_bootstrap.plot(cmap=plt.cm.Blues, values_format="d", ax=axes[0, 0])
    axes[0, 0].set_title("Confusion Matrix (Bootstrap)", fontsize=14)
    axes[0, 0].tick_params(axis='both', labelsize=12)  # Increase tick label size

    disp_smote = ConfusionMatrixDisplay(confusion_matrix=cm_smote, display_labels=np.unique(y))
    disp_smote.plot(cmap=plt.cm.Blues, values_format="d", ax=axes[0, 1])
    axes[0, 1].set_title("Confusion Matrix (SMOTE)", fontsize=14)
    axes[0, 1].tick_params(axis='both', labelsize=12)  # Increase tick label size

    # Plot ROC Curves
    for i in range(y_test_bin_bootstrap.shape[1]):
        fpr_bootstrap, tpr_bootstrap, _ = roc_curve(y_test_bin_bootstrap[:, i], predictions_bin_bootstrap[:, i])
        roc_auc_bootstrap = auc(fpr_bootstrap, tpr_bootstrap)
        axes[1, 0].plot(fpr_bootstrap, tpr_bootstrap, label=f"Class {i} (AUC = {roc_auc_bootstrap:.2f})")

    for i in range(y_test_bin_smote.shape[1]):
        fpr_smote, tpr_smote, _ = roc_curve(y_test_bin_smote[:, i], predictions_bin_smote[:, i])
        roc_auc_smote = auc(fpr_smote, tpr_smote)
        axes[1, 1].plot(fpr_smote, tpr_smote, label=f"Class {i} (AUC = {roc_auc_smote:.2f})")

    # Random guess line for both ROC plots
    for ax in [axes[1, 0], axes[1, 1]]:
        ax.plot([0, 1], [0, 1], "k--", label="Random Guess")
        ax.set_xlabel("False Positive Rate", fontsize=12)
        ax.set_ylabel("True Positive Rate", fontsize=12)
        ax.legend(fontsize=10)

    axes[1, 0].set_title("ROC Curve (Bootstrap)", fontsize=14)
    axes[1, 1].set_title("ROC Curve (SMOTE)", fontsize=14)

    # Final layout adjustments
    plt.tight_layout()
    plt.show()

    # Display Classification Reports
    display_reports_side_by_side(y_test, predictions_bootstrap, predictions_smote)
    
    print("=== Bootstrap Approach ===")
    print(f"Execution Time: {elapsed_bootstrap:.2f} seconds\n")

    print("=== SMOTE Approach ===")
    print(f"Execution Time: {elapsed_smote:.2f} seconds")

#### **First Dataset - Wilt Dataset**

##### **Preprocessing**

In [ ]:
dataset_id = imbalanced_datasets[0]["Dataset ID"]
print("Dataset ID:", dataset_id)
print("Dataset Name:", imbalanced_datasets[0]["Dataset Name"])
data = openml.datasets.get_dataset(dataset_id)

# Transform the dataset to a pandas DataFrame
X, y, _, _ = data.get_data(
    dataset_format="dataframe", target=data.default_target_attribute
)

# Combine X and y for consistent row removal
df = pd.concat([X, y], axis=1)

df.head()

In [ ]:
# Total number of missing values in the dataset
print("Number of missing values:", int(df.isnull().sum().sum()))

In [ ]:
df.describe()

In [ ]:
df["class"].unique()

In [ ]:
df["class"] = df["class"].map({"1": False, "2": True})
df["class"] = df["class"].astype(bool)

df.head()

##### **Evaluating both Random Forest Implementations**

In [ ]:
evaluate_model(df)

#### **Second Dataset - Ozone Level Dataset**

##### **Preprocessing**

In [ ]:
dataset_id = imbalanced_datasets[1]["Dataset ID"]
print("Dataset ID:", dataset_id)
print("Dataset Name:", imbalanced_datasets[1]["Dataset Name"])
data = openml.datasets.get_dataset(dataset_id)

# Transform the dataset to a pandas DataFrame
X, y, _, _ = data.get_data(
    dataset_format="dataframe", target=data.default_target_attribute
)

# Combine X and y for consistent row removal
df = pd.concat([X, y], axis=1)

df.head()

In [ ]:
# Total number of missing values in the dataset
print("Number of missing values:", int(df.isnull().sum().sum()))

In [ ]:
df.describe()

In [ ]:
df["Class"].unique()

In [ ]:
df["Class"] = df["Class"].map({"1": 0, "2": 1})
df["Class"] = df["Class"].astype(bool)

df.head()

##### **Evaluating both Random Forest Implementations**

In [ ]:
evaluate_model(df)

#### **Third Dataset - Climate Model Simulation Crashes Dataset**

##### **Preprocessing**

In [ ]:
dataset_id = imbalanced_datasets[2]["Dataset ID"]
print("Dataset ID:", dataset_id)
print("Dataset Name:", imbalanced_datasets[2]["Dataset Name"])
data = openml.datasets.get_dataset(dataset_id)

# Transform the dataset to a pandas DataFrame
X, y, _, _ = data.get_data(
    dataset_format="dataframe", target=data.default_target_attribute
)

# Combine X and y for consistent row removal
df = pd.concat([X, y], axis=1)

df.head()

In [ ]:
# Total number of missing values in the dataset
print("Number of missing values:", int(df.isnull().sum().sum()))

In [ ]:
df.describe()

In [ ]:
df["outcome"].unique()

In [ ]:
df["outcome"] = df["outcome"].map({"0": True, "1": False})
df["outcome"] = df["outcome"].astype(bool)

df.head()

##### **Evaluating both Random Forest Implementations**

In [ ]:
evaluate_model(df)

#### **Fourth Dataset - PC3 Software Defect Prediction Dataset**

##### **Preprocessing**

In [ ]:
dataset_id = imbalanced_datasets[3]["Dataset ID"]
print("Dataset ID:", dataset_id)
print("Dataset Name:", imbalanced_datasets[3]["Dataset Name"])
data = openml.datasets.get_dataset(dataset_id)

# Transform the dataset to a pandas DataFrame
X, y, _, _ = data.get_data(
    dataset_format="dataframe", target=data.default_target_attribute
)

# Combine X and y for consistent row removal
df = pd.concat([X, y], axis=1)

df.head()

In [ ]:
# Total number of missing values in the dataset
print("Number of missing values:", int(df.isnull().sum().sum()))

In [ ]:
df.describe()

In [ ]:
df["c"].unique()

##### **Evaluating both Random Forest Implementations**

In [ ]:
evaluate_model(df)